In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.neighbors import NearestNeighbors
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [14]:
def resize_and_rescale(image, label):
    image = tf.image.resize(image, (64, 64))
    image = image / 255.0
    return image, label

# 데이터 로드
data_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/novetly_dataset/train'
dataset = image_dataset_from_directory(
    data_dir,
    image_size=(64, 64),
    batch_size=8
)

# 이미지 리사이즈
# dataset = dataset.map(resize_and_rescale)

Found 87 files belonging to 1 classes.


In [15]:
# 인코더 부분
input_img = layers.Input(shape=(64, 64, 3))
x = layers.Flatten()(input_img)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
encoded = layers.Dense(16, activation='relu')(x)

# 디코더 부분
x = layers.Dense(16, activation='relu')(encoded)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(64*64*3, activation='sigmoid')(x)
decoded = layers.Reshape((64, 64, 3))(x)

# Autoencoder 모델 구성
autoencoder = Model(input_img, decoded)

# 모델 컴파일 및 훈련
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(dataset, epochs=10)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
# 잠재 표현 추출
latent_representations = []
for img, _ in dataset.unbatch().as_numpy_iterator():
    encoded_img = autoencoder.encoder(np.expand_dims(img, axis=0)).numpy()
    latent_representations.append(encoded_img[0])
latent_representations = np.array(latent_representations)

# Nearest Neighbors 모델 훈련
nn_model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn_model.fit(latent_representations)
